In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import os
import talib as tl
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn_pandas import DataFrameMapper
%matplotlib inline

In [2]:
def fills(df):
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    return df

In [3]:
aapl = fills(pdr.get_data_google('AAPL', '2000-01-01', '2018-01-01'))
spy = fills(pdr.get_data_google('SPY', '2000-01-01', '2018-01-01'))

In [4]:
oopen = aapl['Open'].values
high = aapl['High'].values
low = aapl['Low'].values
close = aapl['Close'].values

In [5]:
runup252 = aapl['Close'].pct_change(252)
aapl['Run Up 252'] = runup252

In [6]:
beta63 = tl.BETA(close, spy['Close'].values, timeperiod=63)
aapl['Beta 63'] = beta63

In [7]:
ema100 = tl.EMA(close, timeperiod=100)
aapl['EMA 100'] = ema100
aapl['EMA 100'] = aapl['EMA 100'].pct_change()

In [8]:
sma100 = tl.SMA(close, timeperiod=100)
aapl['SMA 100'] = sma100
aapl['SMA 100'] = aapl['SMA 100'].pct_change()

In [9]:
sma_mom100 = aapl['SMA 100'] - aapl['SMA 100'].shift(1)
aapl['SMA MOM 100'] = sma_mom100

In [10]:
sp500_sma100 = tl.SMA(spy['Close'].values, timeperiod=100)
aapl['SP500 SMA 100'] = sp500_sma100
aapl['SP500 SMA 100'] = aapl['SP500 SMA 100'].pct_change()

In [11]:
sp500vola = tl.ATR(spy['High'].values, spy['Low'].values, spy['Close'].values, timeperiod=63)
aapl['SP500 Vola 63'] = sp500vola

In [12]:
sharpe_days = 63
daily_ret = aapl['Close'].pct_change()
sharpe63 = np.sqrt(sharpe_days)*(daily_ret.rolling(sharpe_days).mean()/daily_ret.rolling(sharpe_days).std())
aapl['Sharpe 63'] = sharpe63

In [13]:
vola63 = tl.ATR(high, low, close, timeperiod=63)
aapl['Vola 63'] = vola63

In [14]:
return_days = 2
ret = 100*aapl['Close'].pct_change(return_days).shift(-return_days)
aapl[str(return_days) + ' days future return'] = ret

In [15]:
aapl['Weekday'] = aapl.index.dayofweek.map(lambda x: {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri'}.get(x))

In [16]:
def to_class(val, extreme):
    if val < -extreme:
        return 'Neg'
    elif val > extreme:
        return 'Pos'
    else:
        return 'Neutral'
aapl['Return class'] = aapl[str(return_days) + ' days future return'].apply(lambda val: to_class(val, 2))
aapl['Return class'].head()

Date
2002-02-11    Neutral
2002-02-12    Neutral
2002-02-13        Neg
2002-02-14        Neg
2002-02-15        Neg
Name: Return class, dtype: object

In [ ]:
aapl.head(6)

In [17]:
aapl.tail(6)

,Open,High,Low,Close,Volume,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63,2 days future return,Weekday,Return class
Date,,,,,,,,,,,,,,,,,
2017-12-21,174.17,176.02,174.10,175.01,20949896,0.495045,0.141154,0.001409,0.001530,-0.000043,0.000797,1.454152,1.690945,2.485687,-2.536998,Thu,Neg
2017-12-22,174.68,175.42,174.50,175.01,16349444,0.504945,0.139007,0.001379,0.001094,-0.000436,0.000789,1.442816,1.808430,2.460835,-2.519856,Fri,Neg
2017-12-26,170.80,171.47,169.68,170.57,33185536,0.463869,0.138846,0.000813,0.000917,-0.000177,0.000795,1.429915,1.263734,2.506377,0.298997,Tue,Neutral
2017-12-27,170.10,170.78,169.71,170.60,21498213,0.454887,0.137480,0.000800,0.000868,-0.000049,0.000782,1.418646,1.187278,2.483577,-0.803048,Wed,Neutral
2017-12-28,171.00,171.85,170.48,171.08,16480187,0.465228,0.138857,0.000841,0.000749,-0.000119,0.000785,1.405652,1.294132,2.465901,NaN,Thu,Neutral
2017-12-29,170.52,170.59,169.22,169.23,25999922,0.449756,0.142824,0.000601,0.000558,-0.000191,0.000768,1.413657,1.098579,2.456284,NaN,Fri,Neutral


In [18]:
aapl.drop(['Open', 'High', 'Low', 'Close', 'Volume', str(return_days) + ' days future return'], axis=1, inplace=True)
aapl.dropna(inplace=True)

In [20]:
aapl.drop(aapl[aapl['Return class']=='Neutral'].index, inplace=True)

### Resample umbalanced data and Train-test split

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [22]:
train_index = int(len(aapl) * 0.75)
aapl_train = aapl.iloc[:train_index]
aapl_test = aapl.iloc[train_index:]

In [23]:
# df_majority = aapl_train[aapl_train['Return class']=='Neutral']
# df_minority = aapl_train[aapl_train['Return class']!='Neutral']

# df_minority_upsampled = resample(df_minority, 
#                                  replace=True,
#                                  n_samples=len(df_majority)*2)

# aapl_train = pd.concat([df_majority, df_minority_upsampled])
 
aapl_train['Return class'].value_counts()

Pos    684
Neg    484
Name: Return class, dtype: int64

In [24]:
X_train = aapl_train.drop('Return class', axis=1)
X_test = aapl_test.drop('Return class', axis=1)
y_train = aapl_train['Return class']
y_test = aapl_test['Return class']

In [25]:
trafos = [([col], StandardScaler()) for col in aapl.columns[:-2].tolist()]
trafos = [('Weekday', LabelBinarizer())] + trafos
#trafos += [('Return class', LabelBinarizer())]

In [26]:
trafos

[('Weekday', LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)),
 (['Run Up 252'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Beta 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['EMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SMA MOM 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SP500 SMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SP500 Vola 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Sharpe 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Vola 63'], StandardScaler(copy=True, with_mean=True, with_std=True))]

In [27]:
mapper = DataFrameMapper(trafos, df_out=True, default=None)

In [28]:
X_train = mapper.fit_transform(X_train)
X_test = mapper.transform(X_test)
X_train.head()

,Weekday_Fri,Weekday_Mon,Weekday_Thu,Weekday_Tue,Weekday_Wed,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63
Date,,,,,,,,,,,,,,
2003-02-13,0.0,0.0,1.0,0.0,0.0,-1.634307,-0.382022,-0.844214,-0.645275,-0.245382,-0.257658,0.258473,-1.055020,-1.173638
2003-02-18,0.0,0.0,0.0,1.0,0.0,-1.546605,-0.114126,-0.504778,-0.448507,1.286642,0.015459,0.255168,-0.995181,-1.174291
2003-02-26,0.0,0.0,0.0,0.0,1.0,-1.580868,-0.749259,-0.824916,-0.481397,0.009704,-0.138306,0.217179,-1.187160,-1.176837
2003-02-28,1.0,0.0,0.0,0.0,0.0,-1.464384,-0.775186,-0.619054,-0.350480,0.774360,0.151508,0.209285,-0.823135,-1.178132
2003-03-11,0.0,0.0,0.0,1.0,0.0,-1.651674,-0.587795,-0.922261,-0.775673,-0.753183,-0.319690,0.153622,-0.957087,-1.181819


# ML model

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [30]:
def fit_and_print(grid):
    grid.fit(X_train, y_train)
    best_estimator = grid.best_estimator_
    print('Best estimator:\n', str(best_estimator))
    
    y_pred = best_estimator.predict(X_test)
    print('Accuracy score:', str(accuracy_score(y_test, y_pred)))
    print('Classification report\n', classification_report(y_test, y_pred))
    print('Confusion matrix\n', str(confusion_matrix(y_test, y_pred)))

### K nearest

In [55]:
params = {'n_neighbors': np.arange(5,60,3)}
grid = GridSearchCV(estimator=KNeighborsClassifier(n_jobs=4), param_grid=params, scoring='recall_macro')

In [79]:
# fit_and_print(grid)

### Random Forest

In [78]:
params = {'n_estimators': np.arange(40,80,10),
          'max_features': np.arange(2,8)}
for score in ['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'mutual_info_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'v_measure_score']:
    print('==============', score, '==============')
    try:
        grid = GridSearchCV(estimator=RandomForestClassifier(n_jobs=4), param_grid=params, scoring=score)
        fit_and_print(grid)
    except:
        pass

============== accuracy ==============
Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy score: 0.564102564103
Classification report
              precision    recall  f1-score   support

        Neg       0.59      0.28      0.38       185
        Pos       0.56      0.82      0.67       205

avg / total       0.57      0.56      0.53       390

Confusion matrix
 [[ 51 134]
 [ 36 169]]
============== adjusted_mutual_info_score ==============
Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
   

C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1030: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


============== f1_macro ==============


C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy score: 0.528205128205
Classification report
              precision    recall  f1-score   support

        Neg       0.51      0.26      0.34       185
        Pos       0.54      0.77      0.63       205

avg / total       0.52      0.53      0.49       390

Confusion matrix
 [[ 48 137]
 [ 47 158]]
============== f1_micro ==============
Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=7, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=No

...

Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy score: 0.541025641026
Classification report
              precision    recall  f1-score   support

        Neg       0.53      0.28      0.37       185
        Pos       0.54      0.78      0.64       205

avg / total       0.54      0.54      0.51       390

Confusion matrix
 [[ 52 133]
 [ 46 159]]
============== homogeneity_score ==============
Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=7, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity

C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1030: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


============== precision_macro ==============
Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=70, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy score: 0.564102564103
Classification report
              precision    recall  f1-score   support

        Neg       0.63      0.20      0.30       185
        Pos       0.55      0.89      0.68       205

avg / total       0.59      0.56      0.50       390

Confusion matrix
 [[ 37 148]
 [ 22 183]]
============== precision_micro ==============
Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
       

C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\metrics\classification.py:1030: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


============== recall_macro ==============
Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=70, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy score: 0.558974358974
Classification report
              precision    recall  f1-score   support

        Neg       0.63      0.17      0.26       185
        Pos       0.55      0.91      0.68       205

avg / total       0.59      0.56      0.49       390

Confusion matrix
 [[ 31 154]
 [ 18 187]]
============== recall_micro ==============
Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=6, max_leaf_nodes=None,
            m

### Neural net

In [80]:
from sklearn.neural_network import MLPClassifier

In [81]:
params = {'alpha': [1e-5],
          'hidden_layer_sizes': [(4,2), (5,2), (8,2), (10,8,2), (10, 5, 2)]}

grid = GridSearchCV(estimator=MLPClassifier(max_iter=2000), param_grid=params, scoring='accuracy')

In [82]:
fit_and_print(grid)

Best estimator:
 MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
Accuracy score: 0.566666666667
Classification report
              precision    recall  f1-score   support

        Neg       0.60      0.25      0.36       185
        Pos       0.56      0.85      0.67       205

avg / total       0.58      0.57      0.52       390

Confusion matrix
 [[ 47 138]
 [ 31 174]]
